>># **Ciencia de Datos en Python**
### Proyecto Final
###### *Marco Tulio Escobar Muñoz*

In [ ]:
import numpy as np
import panda as pd
import matplotlib as mt
import seaborn as sns
import scikit-learn as sl

class my_regresion(self, x, y):
    def __init__():
        self.beta0 = []
        self.beta1 = []
        self.x, self.y = [] 
        self.A, self.B, self.C, self.D, self.E, self.F = 0
        self.y = []
        self.archivo = np.load("proyecto_training_data.npy")
        self.dsEntrenamiento = []
        self.dsEntrenamientoNan = []
        self.dsValidacion = []
        self.flMedia = []
        self.flValorMaximo = []
        self.flValorMinimo = []
        self.flRango = []
        self.flDesviacionEstandar = []
        self.long = 0
        self.long81 = 0
        self.long80 = 0
        self.long20 = 0
        self.epochs = 0
        self.print_error = 0
        self.learing_rate = 0.0
        self.matriz = []
        self.ArrVal = []
    
    def GetDataSet(self):
        self.long = len(self.archivo)
        self.long81 = (long * 0.80) + 1
        self.long80 = long * 0.80
        self.long20 = long * 0.20
        self.dsEntrenamiento = archivo[0:long81]
        self.dsValidacion = archivo[long80:]
        
        return self.dsEntrenamiento, self.dsValidacion
    
    def GetExploraionData(self, dsDataSet):
        self.dsEntrenamientoNan = dsDataSet
        self.dsEntrenamientoNan[np.isnan(dsDataSet)] = 0
        self.flMedia = np.nanmean(dsDataSet,axis=1)
        self.flValorMaximo = np.nanmax(dsDataSet,axis=1)
        self.flValorMinimo = np.nanmin(dsDataSet,axis=1)
        self.flRango = np.ptp(dsDataSet,axis=1)
        self.flDesviacionEstandar = np.nanstd(dsDataSet,axis=1)
        
        return self.flMedia, self.flValorMaximo, self.flValorMinimo, self.flRango, self.DesviacionEstandar
    
    def GetHistogram(self, dsDataSet):
        sns.set_theme()
        rug=True, hist=False
        distplot(dsDataSet, rug=True, hist=False)
    
    @staticmethod
    def DivArrCoe(self, dsDataSet):
        #ρ = -1 Correlación perfecta negativa
        #ρ = 0  No existe correlación
        #ρ = +1 Correlación perfecta positiva
        #self.A, self.B, self.C, self.D, self.E, self.F = np.hsplit(self.dsEntrenamiento,6)
        self.A = np.corrcoef(dsDataSet[:,0],dsDataSet[:,1])[1,0]
        self.B = np.corrcoef(dsDataSet[:,0],dsDataSet[:,2])[1,0]
        self.C = np.corrcoef(dsDataSet[:,0],dsDataSet[:,3])[1,0]
        self.D = np.corrcoef(dsDataSet[:,0],dsDataSet[:,4])[1,0]
        self.F = np.corrcoef(dsDataSet[:,0],dsDataSet[:,5])[1,0]
        self.E = max(self.A,self.B,self.C,self.D,self.E)
        if self.E = self.A:
            plt.plot()
            plt.scatter(dsDataSet[:,0],dsDataSet[:,1])
            plt.title('Coeficiente ' + "{:.2f}".format(self.A[0,1]))
            plt.show()
        if self.E = self.B:
            plt.scatter(dsDataSet[:,0],dsDataSet[:,2])
            plt.title('Coeficiente ' + "{:.2f}".format(self.B[0,1]))
            plt.show()
        if self.E = self.C:
            plt.scatter(dsDataSet[:,0],dsDataSet[:,3])
            plt.title('Coeficiente ' + "{:.2f}".format(self.C[0,1]))
            plt.show()
        if self.E = self.D:
            plt.scatter(dsDataSet[:,0],dsDataSet[:,4])
            plt.title('Coeficiente ' + "{:.2f}".format(self.D[0,1]))
            plt.show()
        if self.E = self.F:
            plt.scatter(dsDataSet[:,0],dsDataSet[:,5])
            plt.title('Coeficiente ' + "{:.2f}".format(self.F[0,1]))
            plt.show()
        
        def TrainingModel(self, depX, indY, intEpochs, intPrintError, fltLearningRate):
            self.x = depX, self.y = indY, self.epochs = intEpochs, self.print_error = intPrintError, self.learning_rate = fltLearningRate
            self.ArrVal = np.empty(len(self.x))
            self.ArrVal.fill(1)
            self.matriz = np.append()
            
            
            return print('Número de Iteraciones: {} , error del modelo: {} , error en la iteración: {}'.format(self.epochs, self.print_error))
    